In [6]:
# !wget https://huggingface.co/datasets/nomic-ai/gpt4all-j-prompt-generations/resolve/v1.3-groovy/data/train-00000-of-00004-dd851cdd43f34c01.parquet
# !wget https://huggingface.co/datasets/nomic-ai/gpt4all-j-prompt-generations/resolve/v1.3-groovy/data/train-00001-of-00004-ca093e309275aad2.parquet
# !wget https://huggingface.co/datasets/nomic-ai/gpt4all-j-prompt-generations/resolve/v1.3-groovy/data/train-00002-of-00004-f593963737873d23.parquet
# !wget https://huggingface.co/datasets/nomic-ai/gpt4all-j-prompt-generations/resolve/v1.3-groovy/data/train-00003-of-00004-2978014c5036f638.parquet

In [7]:
from glob import glob

files = glob('*.parquet')
files

['train-00001-of-00004-ca093e309275aad2.parquet',
 'train-00000-of-00004-dd851cdd43f34c01.parquet',
 'train-00002-of-00004-f593963737873d23.parquet',
 'train-00003-of-00004-2978014c5036f638.parquet']

In [8]:
import pandas as pd

dfs = []
for f in files:
    df = pd.read_parquet(f)
    dfs.append(df)

In [9]:
dfs = pd.concat(dfs)[['prompt', 'response']]

In [10]:
html_tags = [
    '<html>',
    '<p>',
    '</a>',
    '<pre>',
    '<code>',
    '<body>',
    '<div>',
]

In [11]:
dfs.iloc[0].to_dict()

{'prompt': 'What are the five major types of synapse?',
 'response': 'The five major types of synapse include axodendritic synapses, axosomatic synapses, axoaxonic synapses, dendrodendritic synapses, and neuromuscular junctions.'}

In [12]:
from tqdm import tqdm

filtered = []
for i in tqdm(range(len(dfs))):
    data = dfs.iloc[i].to_dict()
    if (
        '`' not in data['prompt'] and
        '`' not in data['response'] and
        len(data['prompt']) < 4000 and
        len(data['response']) < 4000 and
        all([t not in data['prompt'] for t in html_tags]) and
        all([t not in data['response'] for t in html_tags])
    ):
        filtered.append(data)

100%|████████████████████████████████| 744469/744469 [00:20<00:00, 35823.40it/s]


In [13]:
len(filtered)

447848

In [ ]:
import json

with open('gpt4all-filtered-noncode.jsonl', 'w') as fopen:
    for d in filtered:
        fopen.write(f'{json.dumps(d)}\n')

In [ ]:
!head -n 100 gpt4all-filtered-noncode.jsonl

In [ ]:
!split -l 20000 -d --additional-suffix=.splitted gpt4all-filtered-noncode.jsonl gpt4all-filtered-noncode.jsonl

In [14]:
filtered = []
for i in tqdm(range(len(dfs))):
    data = dfs.iloc[i].to_dict()
    if (
        ('`' in data['prompt'] or
        '`' in data['response']) and
        len(data['prompt']) < 4000 and
        len(data['response']) < 4000
#         and (any([t in data['prompt'] for t in html_tags]) or
#         any([t in data['response'] for t in html_tags]))
    ):
        filtered.append(data)

100%|████████████████████████████████| 744469/744469 [00:19<00:00, 38800.08it/s]


In [15]:
len(filtered)

108602

In [16]:
filtered

[{'prompt': 'Can you show me a Python function to turn first char to lowercase.',
  'response': "Here's a Python function to turn the first character of a string to lowercase:\n\n```python\n\ndef lowercase_first_letter(s):\n   return s[:1].lower() + s[1:] if s else ''\n```"},
 {'prompt': 'What is a good Python function for comparing integers and strings in Python.',
  'response': 'Python has built in functions for comparing integers and strings. `==` is used to compare values for equality. You can use `type()` to check the data types, and convert them to a common type if needed.'},
 {'prompt': 'What is a good Python program to turn a json into a list?',
  'response': "You can use the built-in `json` module in Python to read a JSON file and convert it into a list using the `load()` method. Here's an example code snippet:\n\n```import json\n\nwith open('file.json') as f:\n    data = json.load(f)\n    my_list = list(data)```"},
 {'prompt': 'Does Python have a method to search for a substr

In [ ]:
with open('gpt4all-filtered-code.jsonl', 'w') as fopen:
    for d in filtered:
        fopen.write(f'{json.dumps(d)}\n')